In [98]:
import json
import codecs
import csv
import pandas as pd

NEO_VER_CSV = 'data/out/neo_ver.csv'
NEO_OUT_JSON = 'data/input/records-full.json'
df = pd.read_csv(NEO_VER_CSV)

def get_csv_data(file_path):
    with open(file_path) as f:
        data = csv.reader(f)
    return data

jdata = json.load(codecs.open(NEO_OUT_JSON, 'r', 'utf-8-sig'))

In [91]:
# Single example of getting a dataframe with packages that have same repo url

dfs = []
df2 = df[df['pkg_name'] == 'glib']
df2 = df2[~df2['repo'].isnull()]
pkgmans = df2['pkgman'].unique()
for pkgman in pkgmans:
    dfs.append(df2[df2['pkgman'] == pkgman].drop_duplicates('repo'))
master_df = pd.concat(dfs)
master_df[master_df.duplicated(subset='repo', keep=False)]

,:ID,pkg_name,pkgman,version,:LABEL,homepage,repo
192520,version17029cargo,glib,cargo,0.0.5,Version,https://github.com/gtk-rs/gtk-rs-core,https://developer.gnome.org/glib
534170,version205chromebrew,glib,chromebrew,2.72.1,Version,https://gitlab.gnome.org/GNOME/glib.git,https://developer.gnome.org/glib
709199,version1875homebrew,glib,homebrew,2.72.1,Version,https://download.gnome.org/sources/glib/2.72/g...,https://developer.gnome.org/glib/
25850731,version625vcpkg,glib,vcpkg,2.70.1,Version,NaN,https://developer.gnome.org/glib/


In [107]:
import math

# Example that iterates the records.json
result = []
TOTAL = len(jdata)
CURRENT = 0
PROGRESS = 0

for r in jdata:

    # Get name pkg and create initial list for dataframes
    name = r["name"]
    frames = []

    CURRENT += 1

    if PROGRESS < math.ceil((CURRENT/TOTAL)*100):
        PROGRESS = math.ceil((CURRENT/TOTAL)*100)
        print(f"Processing {PROGRESS}%  | {CURRENT}/{TOTAL}")

    # For each pkg(name), iterate the pkg managers 
    for pkgman in r["pkgmans"]:

        # Get df from NEO_VER for that pkg and remove rows with NaN in 'repo'
        ndf = df[df['pkg_name'] == name]
        ndf = ndf[~ndf['repo'].isnull()]

        # Get df without duplicates and append to dataframes list
        frames.append(ndf[ndf['pkgman'] == pkgman].drop_duplicates('repo'))


    # Concat all frames and keep only duplucated on repo field. Duplicated would be those who appear < 1
    mdf = pd.concat(frames)
    mdf = mdf[mdf.duplicated(subset='repo', keep=False)]

    # Only add if merged dataframe is not empty
    if not mdf.empty:
        result.append(mdf)

Processing 1%  | 1/12063
Processing 2%  | 121/12063
Processing 3%  | 242/12063
Processing 4%  | 362/12063
Processing 5%  | 483/12063
Processing 6%  | 604/12063
Processing 7%  | 724/12063
Processing 8%  | 845/12063
Processing 9%  | 966/12063
Processing 10%  | 1086/12063
Processing 11%  | 1207/12063
Processing 12%  | 1327/12063
Processing 13%  | 1448/12063
Processing 14%  | 1569/12063
Processing 15%  | 1689/12063
Processing 16%  | 1810/12063
Processing 17%  | 1931/12063
Processing 18%  | 2051/12063
Processing 19%  | 2172/12063
Processing 20%  | 2292/12063
Processing 21%  | 2413/12063
Processing 22%  | 2534/12063
Processing 23%  | 2654/12063
Processing 24%  | 2775/12063
Processing 25%  | 2896/12063
Processing 26%  | 3016/12063
Processing 27%  | 3137/12063
Processing 28%  | 3258/12063
Processing 29%  | 3378/12063
Processing 30%  | 3499/12063
Processing 31%  | 3619/12063
Processing 32%  | 3740/12063
Processing 33%  | 3861/12063
Processing 34%  | 3981/12063
Processing 35%  | 4102/12063
Proce

In [101]:
len(jdata)

12063

In [109]:
fin = pd.concat(result)
fin.to_csv("ne4out.csv", encoding='utf-8', index=False)

In [110]:
len(result)

884